In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
plt.rcParams['axes.unicode_minus'] = False

# 데이터 준비

In [17]:
# load _dataset 함수를 사용하여 데이터프레임으로 변환
df = sns.load_dataset("titanic")
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


# 데이터 탐색

In [18]:
# 데이터 자료형 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [19]:
df.isna().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [20]:
# 결측치가 많은 deck열은 삭제, embarked와 내용이 겹치는 embark_town열은 삭제
rdf = df.drop(["deck", "embark_town"], axis = 1)
rdf.columns.values

array(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'alive', 'alone'],
      dtype=object)

In [21]:
# age 열에 결측치가 있는 모든 행을 삭제(177개 행)
rdf = rdf.dropna(subset=["age"], axis= 0)

In [22]:
len(rdf)

714

In [23]:
# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
most_freq = rdf["embarked"].value_counts(dropna = True).idxmax()
most_freq

'S'

In [24]:
rdf["embarked"].fillna(most_freq, inplace = True)

# 분석에 사용할 속성을 선택

In [26]:
ndf = rdf[["survived", "pclass", "sex", "age", "sibsp", "parch", "embarked"]]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [27]:
# 원핫 인코딩 - 범주형데이터를 인식할 수 있도록 숫자형으로 변환
onehot_sex = pd.get_dummies(ndf["sex"])

In [28]:
onehot_sex

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1
...,...,...
885,1,0
886,0,1
887,1,0
889,0,1


In [29]:
ndf = pd.concat([ndf, onehot_sex], axis = 1)

In [30]:
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male
0,0,3,male,22.0,1,0,S,0,1
1,1,1,female,38.0,1,0,C,1,0
2,1,3,female,26.0,0,0,S,1,0
3,1,1,female,35.0,1,0,S,1,0
4,0,3,male,35.0,0,0,S,0,1


In [31]:
onehot_embarked = pd.get_dummies(ndf["embarked"], prefix = "town")

In [32]:
onehot_embarked.head()

,town_C,town_Q,town_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


In [33]:
ndf = pd.concat([ndf, onehot_embarked], axis = 1)

In [34]:
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male,town_C,town_Q,town_S
0,0,3,male,22.0,1,0,S,0,1,0,0,1
1,1,1,female,38.0,1,0,C,1,0,1,0,0
2,1,3,female,26.0,0,0,S,1,0,0,0,1
3,1,1,female,35.0,1,0,S,1,0,0,0,1
4,0,3,male,35.0,0,0,S,0,1,0,0,1


In [35]:
ndf.drop(["sex", "embarked"], axis = 1, inplace = True)

In [36]:
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


# 데이터셋 분할 - 훈련용 / 테스트용

In [37]:
# 변수 선택
x = ndf.drop("survived", axis = 1) # 독립변수 x
y = ndf["survived"] # 종속변수 y

In [40]:
# train_data와 test_data 구분(7:3 비율)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, stratify = y, random_state = 1)

In [41]:
len(x_train), len(x_test)

(499, 215)

## KNN 분류

In [42]:
np.sqrt(len(x_train))

22.338307903688676

In [44]:
# KNN 분류 모형 객체 생성
knn = KNeighborsClassifier(n_neighbors = 23)

In [45]:
# train data로 모형 학습
knn.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=23)

In [46]:
# test data로 예측
y_pred = knn.predict(x_test)

In [47]:
pd.DataFrame({"ans" : y_test, 
              "pred" : y_pred})

,ans,pred
337,1,0
686,0,0
595,0,0
8,1,0
266,0,0
...,...,...
287,0,0
707,1,0
195,1,0
827,1,1


# 모델 평가

In [48]:
# 모형 성능 평가 - Confusion Matrix 계산
knn_matrix = confusion_matrix(y_test, y_pred)
print(knn_matrix)

[[121   7]
 [ 56  31]]


In [49]:
y_test.value_counts()

0    128
1     87
Name: survived, dtype: int64

## 오차 행렬

<img src = "./image/confusion.png">

- 이진 분류에서 성능 지표로 잘 활용됨
- 분류 모델이 예측을 수행하면서 얼마나 헷갈리고 있는지도 함께 보여주는 지표
    - 이진 분류의 예측 오류가 얼마인지와 더불어 어떠한 유형의 예측 오류가 발생하고 있는지를 함께 나타내는 지표

In [51]:
# 모형 성능 평가 - 평가지표 계산
knn_report = classification_report(y_test, y_pred)

In [52]:
print(knn_report)

              precision    recall  f1-score   support

           0       0.68      0.95      0.79       128
           1       0.82      0.36      0.50        87

    accuracy                           0.71       215
   macro avg       0.75      0.65      0.64       215
weighted avg       0.74      0.71      0.67       215



- 정확도(accuracy)
    - 예측 결과와 실제 값이 동일한 건수 / 전체 데이터 수
    - (TN + TP) /(TN + FP + FN + TP)
    
- 정밀도(precision)
    - 양성으로 예측한 데이터 중 실제로 양성인 데이터의 비율
    - TP / (FP + TP)
    - 정밀도가 더 중요한 지표인 경우는 양성으로 분류된 데이터가 실제로는 양성이 아니라면 업무상 큰 영향이 발생하는 경우
        - 스팸메일 판단 모델 : 일반 메일을 스팸메일로 잘못 분류하면 메일을 아예 받지 못할 수 있어 업무에 차질이 생김
        
- 재현율, 민감도(recall)
    - 실제 양성인 데이터 중 양성으로 예측한 비율
    - TP / (TP + FN)
    - 재현율이 중요한 지표인 경우는 실제 양성 데이터를 음성으로 잘못 판단하게 되면 업무상 큰 영향이 발생하는 경우
        - 예) 암 진단 모델 : 암환자를 음성으로 잘못 판단한다면 생명을 앗아갈 정도로 심각한 문제이기 때문
        
- 특이도(specificity)
    - 실제 음성인 데이터 중 음성으로 예측한 비율
    - TN / (TN + FP)
    
    
- 코로나 검사키트의 경우
    - 민감도가 90% 이상
        - 실제 양성인 사람이 코로나 검사를 하면 양성으로 예측할 확률 90% 이상
        
    - 특이도 99% 이상
        - 실제 음성인 사람이 코로나 검사를 하면 음성으로 예측될 확률 99% 이상

# 모델 고도화

In [53]:
y.value_counts()

0    424
1    290
Name: survived, dtype: int64

In [54]:
ss = StandardScaler()

In [55]:
scaled_train = ss.fit_transform(x_train)
scaled_test = ss.transform(x_test)

In [57]:
knn = KNeighborsClassifier(n_neighbors = 23)

In [58]:
knn.fit(scaled_train, y_train)

KNeighborsClassifier(n_neighbors=23)

In [59]:
y_pred = knn.predict(scaled_test)

In [60]:
pd.DataFrame({"ans" : y_test, "pred" : y_pred})

,ans,pred
337,1,1
686,0,0
595,0,0
8,1,1
266,0,0
...,...,...
287,0,0
707,1,0
195,1,1
827,1,0


In [61]:
knn_matrix = confusion_matrix(y_test, y_pred)
print(knn_matrix)

[[115  13]
 [ 32  55]]


In [62]:
knn_report = classification_report(y_test, y_pred)
print(knn_report)

              precision    recall  f1-score   support

           0       0.78      0.90      0.84       128
           1       0.81      0.63      0.71        87

    accuracy                           0.79       215
   macro avg       0.80      0.77      0.77       215
weighted avg       0.79      0.79      0.79       215

